# cTraderFix Jupyter Sample

In this Jupyter notebook we will use the Python package "ctrader-fix" to interact with cTrader FIX API.

Let's start.

If you haven't already installed the "ctrader-fix" package, run the next code cell to install it via pip:

In [ ]:
!pip install ctrader-open-api

Then we have to import all necessary types:

In [ ]:
from twisted.internet import reactor
import json
from ctrader_fix import *
import datetime

Now we use the "config-trade.json" config file to get your API credentials.
Be sure to populate it with your API credentials before running next cell:

In [ ]:
# you can use two separate config files for QUOTE and TRADE
with open("config-trade.json") as configFile:
    config = json.load(configFile)

Then we will create a client based on our config:

In [ ]:
client = Client(config["Host"], config["Port"], ssl = config["SSL"])

We will use ther below "send" method for sending our message:

In [ ]:
def send(request):
    diferred = client.send(request)
    diferred.addCallback(lambda _: print("\nSent: ", request.getMessage(client.getMessageSequenceNumber()).replace("", "|")))

Now let's set and client call backs that will be used by client after it got message received/connected/disconnected:

In [ ]:
def onMessageReceived(client, responseMessage): # Callback for receiving all messages
    print("\nReceived: ", responseMessage.getMessage().replace("", "|"))
    # We get the message type field value
    messageType = responseMessage.getFieldValue(35)
    # we send a security list request after we received logon message response
    if messageType == "A":
        securityListRequest = SecurityListRequest(config)
        securityListRequest.SecurityReqID = "A"
        securityListRequest.SecurityListRequestType = 0
        send(securityListRequest)
    # After receiving the security list we send a market order request by using the security list first symbol
    elif messageType == "y":
        newOrderSingle = NewOrderSingle(config)
        newOrderSingle.ClOrdID = "B"
        # We use getFieldValue to get all symbol IDs, it will return a list in this case
        # because the symbol ID field is repetitive
        symboldIds = responseMessage.getFieldValue(55)
        newOrderSingle.Symbol = symboldIds[0]
        newOrderSingle.Side = 1
        newOrderSingle.OrderQty = 1000
        newOrderSingle.OrdType = 1
        newOrderSingle.Designation = "From Jupyter"
        send(newOrderSingle)
    # after receiving the new order request response we stop the reactor
    # And we will be disconnected from API
    elif messageType == "8" or messageType == "j":
        print("We are done, stopping the reactor")
        reactor.stop()

def disconnected(client, reason): # Callback for client disconnection
    print("\nDisconnected, reason: ", reason)

def connected(client): # # Callback for client connection
    print("Connected")
    logonRequest = LogonRequest(config)
    send(logonRequest)
    
# Setting client callbacks
client.setConnectedCallback(connected)
client.setDisconnectedCallback(disconnected)
client.setMessageReceivedCallback(onMessageReceived)

The last step is to run our client service, it will run inside Twisted reactor loop asynchronously:

In [ ]:
# Starting the client service
client.startService()
# Run Twisted reactor, we imported it earlier
reactor.run()